# NLP Kaggle Tutorial (Phase 2)


The [actual tutorial](https://www.kaggle.com/c/word2vec-nlp-tutorial#part-1-for-beginners-bag-of-words) was developed by **Angela Chapman** during her summer 2014 internship at Kaggle.  
Full credits to her!

This notebook by Prashant Brahmbhatt                      

### Introducing Distributed Word Vectors

In [1]:
import re
import pandas as pd
import numpy as np
import nltk
from bs4 import BeautifulSoup
from nltk.corpus import stopwords

**Word2Vec** does not need labels in order to create meaningful representations.  
This is useful, since most data in the real world is unlabeled. If the network is given enough training data (tens of billions of words), it produces word vectors with intriguing characteristics.  
Words with similar meanings appear in clusters, and clusters are spaced such that some word relationships, such as analogies, can be reproduced using vector math.   
**Example**  
"king - man + woman = queen."

______________________

Check out [Google's code, writeup, and the accompanying papers](https://code.google.com/archive/p/word2vec/). [This presentation](https://docs.google.com/file/d/0B7XkCwpI5KDYRWRnd1RzWXQ2TWc/edit) is also helpful. The original code is in C, but it has since been ported to other languages, including Python.  
Recent work out of Stanford has also applied deep learning to sentiment analysis; their code is available in Java. However, their approach, which relies on sentence parsing, cannot be applied in a straightforward way to paragraphs of arbitrary length.

__________________

## Preparing to Train the model

First, we read in the data with pandas, we now use data with 50,000 additional reviews with no labels. 
When we built the **Bag of Words** model previously extra unlabeled training reviews were not useful.  
However, since **Word2Vec** can learn from unlabeled data, these extra 50,000 reviews can now be used.

In [2]:
train = pd.read_csv('kaggleNLP/labeledTrainData.tsv',sep = '\t' , quoting = 3)
test = pd.read_csv('kaggleNLP/testData.tsv', sep = '\t', quoting = 3)
unlabeled = pd.read_csv('kaggleNLP/unlabeledTrainData.tsv', sep = '\t', quoting = 3 )

In [3]:
print("Labeled Train reviews {}\nLabeled Test reviews {}\nUnlabeled reviews {}".
      format(train['review'].size, test['review'].size, unlabeled['review'].size))

Labeled Train reviews 25000
Labeled Test reviews 25000
Unlabeled reviews 50000


First, to train **Word2Vec** it is better not to remove **stop words** because the algorithm relies on the broader context of the sentence in order to produce high-quality word vectors.  
For this reason, we will make stop word removal optional in the functions below. It also might be better not to remove numbers, but its up to the programmer.

In [4]:
def review_to_wordlist(review, remove_stopwords = False):
    # Function to convert a document to a sequence of words
    
    #Removing HTML
    review_text = BeautifulSoup(review).get_text()
    
    #Removing Non Letters
    review_text = re.sub('[^a-zA-Z]',' ',review_text)
    
    #Convert words to lowercase and split
    words = review_text.lower().split()
    
    #Optionally removing stopwords
    if remove_stopwords:
        words = [w for w in words if w not in set(stopwords.words('english'))]
    #Returing the list of words
    return (words)

We want a specific input format. **Word2Vec** expects single sentences, each one as a list of words. In other words, the input format is a **list of lists**.

Spltting the paragraph is not very strightforward. There are all kinds of gotchas in natural language. English sentences can end with **"?"**, **"!"**, **"""**, or **"."**, among other things, and spacing and capitalization are not reliable guides either.  
For this reason, we use **NLTK's punkt tokenizer** for sentence splitting.

_____

In [5]:
#nltk.download()
# Load the punkt tokenizer
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

_______

Defining a function to split a review into parsed sentences

In [6]:
def review_to_sentences(review , tokenizer, remove_stopwords = False):
    # Function to split a review into parsed sentences. Returns a 
    # list of sentences, where each sentence is a list of words

    # Use the NLTK tokenizer to split the paragraph into sentences
    raw_sentences = tokenizer.tokenize(review.strip())

    #Looping over each sentence
    sentences = []

    #If sentence is empty skip otherwie call review_to_wordlist
    for sentence in raw_sentences:
        if len(sentence) >0:
            sentences.append(review_to_wordlist(sentence, remove_stopwords))
        
    return sentences

Now we can apply this function to prepare our data for input to **Word2Vec**

In [7]:
sentences = []
print('Parsing sentences from Reviews')
for rev in train['review']:
    sentences += review_to_sentences(rev, tokenizer)
    
print('Parsing sentences for unlabeled data')
for rev in unlabeled['review']:
    sentences += review_to_sentences(rev, tokenizer)

Parsing sentences from Reviews


C:\Users\prash\Anaconda3\envs\tensorflow\lib\site-packages\bs4\__init__.py:273: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
C:\Users\prash\Anaconda3\envs\tensorflow\lib\site-packages\bs4\__init__.py:273: UserWarning: "b'...'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
C:\Users\prash\Anaconda3\envs\tensorflow\lib\site-packages\bs4\__init__.py:336: UserWarning: "http://www.happierabroad.com"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup


Parsing sentences for unlabeled data


C:\Users\prash\Anaconda3\envs\tensorflow\lib\site-packages\bs4\__init__.py:336: UserWarning: "http://www.archive.org/details/LovefromaStranger"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
C:\Users\prash\Anaconda3\envs\tensorflow\lib\site-packages\bs4\__init__.py:336: UserWarning: "http://www.loosechangeguide.com/LooseChangeGuide.html"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
C:\Users\prash\Anaconda3\envs\tensorflow\lib\site-packages\bs4\__init__.py:273: UserWarning: "b'... ...'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.


We can see we have got warnings which are about the URLs in the sentences which are not a big trouble. Although we may want to remove these too.

___________

**Unconfirmed** (don't Read)

A minor detail to note is the difference between the **"+="** and **"append"** when it comes to Python lists. 
In many applications the two are interchangeable, but here they are not. 
When appending a list of lists to another list of lists, **"append"** will only append the first list; you need to use **"+="** in order to join all of the lists at once.

_____

## Training and Saving the Model

There are a number of parameter choices that affect the run time and the quality of the final model that is produced.  
For details on the algorithms below, see the word2vec [API documentation](radimrehurek.com/gensim/models/word2vec.html) as well as the [Google documentation](https://code.google.com/archive/p/word2vec/).

- **Architecture**: Architecture options are skip-gram (default) or continuous bag of words. We found that skip-gram was very slightly slower but produced better results.


- **Training algorithm**: Hierarchical softmax (default) or negative sampling. For us, the default worked well. 


- **Downsampling of frequent words**: The Google documentation recommends values between .00001 and .001. For us, values closer 0.001 seemed to improve the accuracy of the final model.


- **Word vector dimensionality**: More features result in longer runtimes, and often, but not always, result in better models. Reasonable values can be in the tens to hundreds; we used 300.


- **Context / window size**: How many words of context should the training algorithm take into account? 10 seems to work well for hierarchical softmax (more is better, up to a point).


- **Worker threads**: Number of parallel processes to run. This is computer-specific, but between 4 and 6 should work on most systems.


- **Minimum word count**: This helps limit the size of the vocabulary to meaningful words. Any word that does not occur at least this many times across all documents is ignored. Reasonable values could be between 10 and 100. In this case, since each movie occurs 30 times, we set the minimum word count to 40, to avoid attaching too much importance to individual movie titles. This resulted in an overall vocabulary size of around 15,000 words. Higher values also help limit run time.
